In [ ]:
# Perform MultiBlast and Find Shared Contigs
# __Objective__ : The objective of this notebook is to enable blasting multiple sequences which are preseumed to co-locate in a biosynthetig gene cluster (BSG) and then find contigs which intersect multiple blast results, demonstrating a BSG is likely to exist in that cluster, and improve confidence these genes perform the desired activity, enabling their use in HMM building and gene diversity searches or directly in pathway design.
# __Context:__ Many of the pathways we desire to investigate within ZED exist within biosynthetic gene clusters.  While tools like Antismash and others exist for the investigation of BSG's some limitations arise (List them?).  
# In select cases, BSG's can be discovered/explored by looking for contigs that contain a number of desired functionalities in close proximity.  When this occurs, we have increased confidence that these genes work together within that cluster, and thus have the desired activity, versus genes which are unclustered and may perform similar, but distinct, activities.
# __Steps__
#    1. Load necessary dependencies
#    2. Load the blast query file and make the API call to NCBI
#    3. Parse the results from blast
# __Notes__
#    Much of the base code was taken from this tutorial to enable access to NCBI: https://www.tutorialspoint.com/biopython/biopython_overview_of_blast.htm
#    Additionally; the docs on the Blast.Record provided the necessary information to be able to pull 
#    https://biopython.org/DIST/docs/api/Bio.Blast.Record-pysrc.html

In [ ]:
# 1. Import necessary dependencies

In [1]:
# Leveraging biopython here as much as possible as they have pre-built wrappers for making these calls
from Bio.Blast import NCBIWWW  # Make the API request
from Bio.Blast import NCBIXML  # Parse the NCBI XML request body

# Import pandas for data manipulation
import pandas as pd

In [ ]:
# 2. Load the Query File and Run the Blast

In [4]:
# Load the fasta file containing anchor sequences loaded into the current directory
sequence_data = open("sequence.fasta").read()
# Inspect for good measure
print(sequence_data)

>XM_748623.1 Aspergillus fumigatus Af293 transaldolase (AFUA_5G09230), partial mRNA
ATGTCTTCCGCTCTCGAACAGCTCAAGGCCACTGGCACCGTTGTTGTCTGCGACTCTGGTGACTTTGCCACCATTGGCAA
GTACAAGCCGCAGGATGCTACCACCAACCCTTCCTTGATCCTGGCTGCTTCTAAGAAGCCCGAGTACGCCAGCCTGATCG
ACGCTGCCGTCCAGAAGGGCAAGAAGGAGGGCAAGACTCTTGATGAGCAGGTCGACGCTACCCTCGACAACCTTCTCGTT
GAGTTCGGCAAGAAGATCCTCGAGATCATCCCCGGCAAGGTCTCCACTGAGGTTGATGCCCGGTTCTCCTTCGACACCCA
GGCTTCCGTCGACAAGGCCCTTCACATCGTCAAGCTCTACGAGCAACAAGGCATCTCCAAGGACCGTATTCTTATCAAGA
TCGCCTCCACCTGGGAGGGTATCAAGGCCGCCCACATCCTGCAGTCTCAGCACGGCATCAACTGCAACCTGACCCTCATG
TTCTCCCTTGTCCAGGCCATTGCTGCTGCTGAGGCTGGTGCTTTCCTCATCTCCCCCTTCGTCGGCCGTATCCTGGACTG
GTACAAGGCTGCGCACAAGCGCGACTTCAGCCCCGAGGAGGATCCCGGTGTCAAGTCCGTTCAGAGCATTTTCAACTACT
ACAAGAAGCATGGGTACAAGACCATCGTCATGGGAGCTTCCTTCCGTAACACCGGCGAGATCACGGAGTTGGCTGGCTGT
GATTACCTGACCATTTCGCCCAACCTACTCGAGGAGTTGTACAACTCGACAGCCTCCGTCCCCAAGAAGCTCGATGCCGC
CAGCGCCGCCAGCCTCGACATCCCCAAACGCAGCTACATCAACGACGAGGCTGCCTTCCGCTTCGACTTCAACGAGGAGG
CTATGGCCGTTGAGAAGCTGCGTGA

In [5]:
# Perform the blast (This can take a while)
result_handle = NCBIWWW.qblast("tblastx", "nt", sequence_data) 

In [6]:
# Save the results locally as an intermediate step as you can only read once using this function
with open('results.xml', 'w') as save_file:
    blast_results = result_handle.read() 
    save_file.write(blast_results)

In [ ]:
# 3. Parse The Results From Blast

In [7]:
E_VALUE_THRESH = 1e-5 
for i, record in enumerate(NCBIXML.parse(open("results.xml"))): 
     if record.alignments: 
        query_name = record.query.split(" ")[0]
        # Clear lists to hold info
        desc_dict_list = []
        align_dict_list = []
        # QUERY SEQUENCE IS KEY BECAUSE THESE ARE SUB TO QUERY SEQ's
        # Parse relevant information from the description (bit score and e value for each hit)
        for description in record.descriptions:
            if description.e < E_VALUE_THRESH: 
                desc_dict_list.append({query_name + "_e_value":description.e,query_name + "_bit_score":description.bits,"title":description.title[:100].split("|")[4],"accession_key":description.title[:100].split("|")[3]})
                # How do I track which query it goes to
        # Parse relevant information from the alignments (namely start location on the contig  so we can use genomic context later)
        for align in record.alignments: 
            for hsp in align.hsps: 
                if hsp.expect < E_VALUE_THRESH: 
                    align_dict_list.append({query_name + "_subject_start":hsp.sbjct_start,query_name + "_length":hsp.align_length,"title":align.title[:100].split("|")[4]})
        # if not the first sequence then merge the new pandas dataframe with the old one in order to create a master dataframe
        if i == 0:
            # Create pandas dataframe from information
            df_blast_results = pd.merge(pd.DataFrame(desc_dict_list), pd.DataFrame(align_dict_list), on= "title",
                     how =  'left').drop_duplicates(subset ="title", keep = 'first') 
        else:
            # Create pandas dataframe from information
            df_temp_results = pd.merge(pd.DataFrame(desc_dict_list), pd.DataFrame(align_dict_list), on="title",
                     how =  'left').drop_duplicates(subset ="title", keep = 'first') 
            df_blast_results = pd.merge(df_blast_results, df_temp_results, on="title",how =  'outer')
# LATER, IT WOULD BE GOOD TO PULL 10K OF SEQUENCES SURROUNDING, PULL PROTEINS FROM IT, AND THEN ALL-VS-ALL BLAST TO GROUP INTO SIMILAR ACTIVITIES/FAMILIES

In [8]:
df_blast_results

,XM_748623.1_e_value,XM_748623.1_bit_score,title,accession_key_x,XM_748623.1_subject_start,XM_748623.1_length,XM_747627.1_e_value,XM_747627.1_bit_score,accession_key_y,XM_747627.1_subject_start,XM_747627.1_length
0,0.0,1759.57,Aspergillus fumigatus Af293 transaldolase (AF...,XM_748623.1,1.0,975.0,NaN,NaN,NaN,NaN,NaN
1,0.0,1646.86,Aspergillus fischeri NRRL 181 transaldolase p...,XM_001259766.1,1.0,975.0,NaN,NaN,NaN,NaN,NaN
2,0.0,1583.74,Aspergillus lentulus transaldolase (IFM58399_...,XM_033561900.1,1.0,975.0,NaN,NaN,NaN,NaN,NaN
3,0.0,1529.64,Aspergillus thermomutatus hypothetical protei...,XM_026757392.1,1.0,975.0,NaN,NaN,NaN,NaN,NaN
4,0.0,1526.94,Aspergillus novofumigatus IBT 16806 transaldo...,XM_024828790.1,211.0,936.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
93,NaN,NaN,Aspergillus nidulans FGSC A4 hypothetical pro...,NaN,NaN,NaN,0.0,2152.71,XM_653200.1,1.0,2080.0
94,NaN,NaN,Penicillium digitatum Pd1 Transketolase TktA ...,NaN,NaN,NaN,0.0,2113.93,XM_014680637.1,175.0,2057.0
95,NaN,NaN,Rasamsonia emersonii CBS 393.64 Transketolase...,NaN,NaN,NaN,0.0,2039.99,XM_013469015.1,1.0,2080.0
96,NaN,NaN,Talaromyces atroroseus Transketolase (UA08_00...,NaN,NaN,NaN,0.0,1940.81,XM_020260721.1,1.0,2049.0


In [10]:
df_blast_results.sort_values(by=[query_name + "_e_value"])

,XM_748623.1_e_value,XM_748623.1_bit_score,title,accession_key_x,XM_748623.1_subject_start,XM_748623.1_length,XM_747627.1_e_value,XM_747627.1_bit_score,accession_key_y,XM_747627.1_subject_start,XM_747627.1_length
15,0.0,1182.490,Penicilliopsis zonata CBS 506.65 hypothetical...,XM_022722497.1,57.0,965.0,0.0,2213.12,XM_022722052.1,261.0,2054.0
75,NaN,NaN,Aspergillus brunneoviolaceus CBS 621.78 trans...,NaN,NaN,NaN,0.0,2413.29,XM_025587880.1,247.0,2055.0
76,NaN,NaN,Aspergillus aculeatinus CBS 121060 transketol...,NaN,NaN,NaN,0.0,2408.78,XM_025646084.1,282.0,2055.0
77,NaN,NaN,Aspergillus heteromorphus CBS 117.55 transket...,NaN,NaN,NaN,0.0,2404.28,XM_025541247.1,50.0,2055.0
78,NaN,NaN,Aspergillus homomorphus CBS 101889 transketol...,NaN,NaN,NaN,0.0,2399.77,XM_025701095.1,177.0,2055.0
...,...,...,...,...,...,...,...,...,...,...,...
45,0.0,991.336,Colletotrichum graminicola M1.001 transaldola...,XM_008100829.1,1.0,975.0,NaN,NaN,NaN,NaN,NaN
46,0.0,988.631,Talaromyces marneffei ATCC 18224 transaldolas...,XM_002147393.1,427.0,975.0,NaN,NaN,NaN,NaN,NaN
47,0.0,985.926,Colletotrichum orchidophilum transaldolase (C...,XM_022613267.1,1.0,975.0,NaN,NaN,NaN,NaN,NaN
48,0.0,983.221,Penicillium chrysogenum Wisconsin 54-1255 hyp...,XM_002568649.1,1.0,976.0,NaN,NaN,NaN,NaN,NaN
